<h1><center><font size="6">CNN with Keras for Fashion MNIST</font></center></h1>


<img src="https://kaggle2.blob.core.windows.net/datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-card.png" width="400"></img>


# <a id='0'>Content</a>

- <a href='#1'>Introduction</a>  
- <a href='#2'>Load packages</a>  
- <a href='#3'>Read and prepare the data</a>  
- <a href='#4'>Data exploration</a>
- <a href='#5'>Prepare the model</a>  
- <a href='#6'>Prediction accuracy</a>  
- <a href='#7'>Conclusions</a>
- <a href='#8'>References</a>

# <a id="1">Introduction</a>  


## Dataset

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.


## Content

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total.   

Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255.   

The training and test data sets have 785 columns.   

The first column consists of the class labels (see above), and represents the article of clothing. 

The rest of 784 columns (1-785) contain the pixel-values of the associated image.

# <a id="2">Load packages</a>

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D

## Parameters

In [16]:
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10

#Model
NO_EPOCHS = 75
BATCH_SIZE = 128

IS_LOCAL = False

import os

if(IS_LOCAL):
    PATH="../input/fashionmnist/"
else:
    PATH="../input/"
print(os.listdir(PATH))

['train-images-idx3-ubyte', 'train-labels-idx1-ubyte', 't10k-labels-idx1-ubyte', 'fashion-mnist_train.csv', 't10k-images-idx3-ubyte', 'fashion-mnist_test.csv']


# <a id="3">Read and prepare the data</a>

In [6]:
img_rows, img_cols = IMG_ROWS, IMG_COLS
num_classes = NUM_CLASSES

# data preprocessing
def data_preprocessing(raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

train_size = TRAIN_SIZE
train_file = PATH+"fashion-mnist_train.csv"
test_file  = PATH+"fashion-mnist_test.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(train_file)

# prepare the data
X, y = data_preprocessing(train_data)
X_test, y_test = data_preprocessing(test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)


# <a id="4">Data exploration</a>

This section is not developed yet.

# <a id="3">Prepare the model</a>

In [9]:
# Model
model = Sequential()
# use strides = 2 (when applying the convolution, step by 2, not by 1 the kernel)
model.add(Conv2D(30, kernel_size=(3, 3),
                 strides=2,
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(MaxPooling2D((2, 2)))
# Add dropouts to the model
model.add(Dropout(0.25))
model.add(Conv2D(30, 
                 kernel_size=(3, 3), 
                 strides=2, 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add dropouts to the model
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

## Show the model

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 13, 13, 30)        300       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 30)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 30)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 30)          8130      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 30)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 1, 30)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30)                0         
__________

## Run the model

In [18]:
history = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=NO_EPOCHS,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/75
48000/48000 [==============================] - 3s 70us/step - loss: 0.6582 - acc: 0.7616 - val_loss: 0.5469 - val_acc: 0.7986
Epoch 2/75
48000/48000 [==============================] - 3s 72us/step - loss: 0.6319 - acc: 0.7713 - val_loss: 0.5187 - val_acc: 0.8100
Epoch 3/75
48000/48000 [==============================] - 3s 72us/step - loss: 0.6113 - acc: 0.7783 - val_loss: 0.4985 - val_acc: 0.8192
Epoch 4/75
48000/48000 [==============================] - 4s 73us/step - loss: 0.5937 - acc: 0.7841 - val_loss: 0.4860 - val_acc: 0.8209
Epoch 5/75
48000/48000 [==============================] - 3s 71us/step - loss: 0.5798 - acc: 0.7896 - val_loss: 0.4790 - val_acc: 0.8227
Epoch 6/75
48000/48000 [==============================] - 3s 72us/step - loss: 0.5648 - acc: 0.7934 - val_loss: 0.4626 - val_acc: 0.8274
Epoch 7/75
48000/48000 [==============================] - 3s 71us/step - loss: 0.5565 - acc: 0.7988 - val_loss: 0.4556 - val_acc

48000/48000 [==============================] - 3s 71us/step - loss: 0.4472 - acc: 0.8348 - val_loss: 0.3908 - val_acc: 0.8502
Epoch 61/75
48000/48000 [==============================] - 3s 71us/step - loss: 0.4487 - acc: 0.8347 - val_loss: 0.3819 - val_acc: 0.8595
Epoch 62/75
48000/48000 [==============================] - 3s 71us/step - loss: 0.4455 - acc: 0.8337 - val_loss: 0.3806 - val_acc: 0.8611
Epoch 63/75
48000/48000 [==============================] - 3s 71us/step - loss: 0.4501 - acc: 0.8336 - val_loss: 0.3811 - val_acc: 0.8604
Epoch 64/75
48000/48000 [==============================] - 3s 71us/step - loss: 0.4456 - acc: 0.8356 - val_loss: 0.3840 - val_acc: 0.8571
Epoch 65/75
48000/48000 [==============================] - 3s 72us/step - loss: 0.4422 - acc: 0.8356 - val_loss: 0.3882 - val_acc: 0.8562
Epoch 66/75
48000/48000 [==============================] - 3s 72us/step - loss: 0.4437 - acc: 0.8367 - val_loss: 0.3782 - val_acc: 0.8597
Epoch 67/75
48000/48000 [=====================

### Evaluate the model score

In [19]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3542315003236135
Test accuracy: 0.8670166666666667


# <a id="6">Prediction accuracy</a>

In [20]:
#get the predictions for the test data
predicted_classes = model.predict_classes(X_test)

#get the indices to be plotted
y_true = test_data.iloc[:, 0]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]



target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

             precision    recall  f1-score   support

    Class 0       0.87      0.81      0.83      6000
    Class 1       1.00      0.95      0.97      6000
    Class 2       0.81      0.80      0.81      6000
    Class 3       0.87      0.90      0.88      6000
    Class 4       0.82      0.67      0.74      6000
    Class 5       0.97      0.94      0.96      6000
    Class 6       0.58      0.74      0.65      6000
    Class 7       0.94      0.92      0.93      6000
    Class 8       0.96      0.97      0.97      6000
    Class 9       0.93      0.97      0.95      6000

avg / total       0.87      0.87      0.87     60000



# <a id="7">Conclusions</a>

The best accuracy is obtained for Class 1, followed by Class 5, Class 8 and 7. Worst accuracy is for Class 6.

The recall is highest for Class 8, 9 and 1 and smallest for Class 4.

f1-score is highest for Class 8 and Class 1 and smallest for Class 5 and Class 4.

# <a id="8">References</a>

[1] Fashion MNIST, An MNIST-like dataset of 70,000 28x28 labeled fashion images, https://www.kaggle.com/zalando-research/fashionmnist  
[2] DanB, CollinMoris, Deep Learning From Scratch, https://www.kaggle.com/dansbecker/deep-learning-from-scratch  
[3] DanB, Dropout and Strides for Larger Models, https://www.kaggle.com/dansbecker/dropout-and-strides-for-larger-models  
[4] BGO, CNN with Keras, https://www.kaggle.com/bugraokcu/cnn-with-keras  
